# Import packages

In [ ]:
# # if using google drive
# %cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Run overnight

In [ ]:
# Predict TAFT vs GUAT
for monkey_name in ['monkey_Bruno', 'monkey_Schro']:
    cgtxs = GUAT_vs_TAFT_x_sessions_class.GUATandTAFTacrossSessionsClass()
    combd_GUAT_x_df, combd_TAFT_x_df = cgtxs.streamline_getting_combd_GUAT_or_TAFT_x_df(monkey_name=monkey_name)

# Predict num_stops
combined_info_exists_ok = True
for monkey_name in ['monkey_Bruno', 'monkey_Schro']:
    gc_kwargs_temp = helper_GUAT_vs_TAFT_class.gc_kwargs.copy()
    gc_kwargs_temp['num_old_ff_per_row'] = 2
    gc_kwargs_temp['num_new_ff_per_row'] = 3

    gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs_temp, monkey_name=monkey_name)
    gas.retrieve_or_make_combined_info(gc_kwargs_temp, combined_info_exists_ok=combined_info_exists_ok, 
                                    traj_df_exist_in_GUAT_store_ok=True)

# predict TAFT vs GUAT

## data across sessions

In [ ]:
monkey_name='monkey_Bruno'
cgtxs = GUAT_vs_TAFT_x_sessions_class.GUATandTAFTacrossSessionsClass()
# now, put the data into a GUATvsTAFTclass instance for downstream analysis
cgt = GUAT_vs_TAFT_class.GUATvsTAFTclass()
cgt.GUAT_x_df, cgt.TAFT_x_df = cgtxs.streamline_getting_combd_GUAT_or_TAFT_x_df(monkey_name=monkey_name)

## data for one session

In [ ]:
exists_ok = True
cgt = GUAT_vs_TAFT_class.GUATvsTAFTclass(ref_point_mode='time', 
                                            raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330',
                                            ref_point_value=-1.5,)
cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='TAFT', exists_ok=exists_ok)
cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='GUAT', exists_ok=exists_ok)


In [ ]:
# # test and see what is taking so long in running a function
# import cProfile

# cProfile.run("cgt.streamline_getting_GUAT_or_TAFT_x_df(GUAT_or_TAFT='GUAT', exists_ok=False)", sort='cumtime')

# #ncalls  tottime  percall  cumtime  percall

# ML

## select features

In [ ]:
cgt.get_x_and_y_var_df()

eye_keywords = ("eye", "gaze", "LD", "RD")
eye_cols = [col for col in cgt.x_var_df.columns if any(k in col for k in eye_keywords)]
cgt.x_var_df = cgt.x_var_df.drop(columns=eye_cols)


from neural_data_analysis.neural_analysis_tools.model_neural_data import design_bases
x_var_df_reduced = design_bases.reduce_df(cgt.x_var_df)

ml_inst = ml_methods_class.MlMethods(x_var_df=x_var_df_reduced,
                                     y_var_df=cgt.y_var_df)

## check vif

In [ ]:
# # can skip this if only wanting ML results
# pd.set_option('display.max_rows', 100)
# ml_inst.use_vif(ml_inst.x_var_df)
# features_w_big_vif = ml_inst.vif_df[ml_inst.vif_df['vif'] > 100].feature.values
# #ml_inst.x_var_df = ml_inst.x_var_df.drop(columns=features_w_big_vif)
# ml_inst.vif_df.head(20)


In [ ]:
# specific_columns = ml_inst.vif_df[ml_inst.vif_df["VIF"] > 2000].feature.values
# ml_inst.show_correlation_heatmap(specific_columns=specific_columns)
# ml_inst.show_correlation_heatmap()

## run ML

In [ ]:
ml_inst.x_var_df.columns[3]

In [ ]:
ml_inst.use_ml_model_for_classification(ml_inst.x_var_df, ml_inst.y_var_df)


# model's feature selection

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(ml_inst.x_var_df, ml_inst.y_var_df)
important = model.coef_[0] != 0
X_new = ml_inst.x_var_df.loc[:, important]
X_new.columns


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
sfs = SequentialFeatureSelector(model, n_features_to_select=10, direction="forward")
X_new = sfs.fit_transform(ml_inst.x_var_df, ml_inst.y_var_df.values.ravel())


In [ ]:
X_new

In [ ]:

# Boolean mask of selected features
mask = sfs.get_support()

# Names of selected features
selected_features = ml_inst.x_var_df.columns[mask]

print("Selected features:")
print(selected_features)

## grad_boosting (so that we can see feature importance)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ml_inst.x_var_df, ml_inst.y_var_df, test_size=0.2, random_state=42)

# Define the model
model = GradientBoostingClassifier(
    learning_rate=0.05, max_depth=7, max_features='sqrt',
    min_samples_leaf=2, min_samples_split=7,
    n_estimators=500, subsample=0.5
)


# Fit the model
model.fit(X_train, y_train)

# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': ml_inst.x_var_df.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 18))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df)
plt.title('Feature Importances')
plt.show()

# Determine significant features (e.g., importance > 0.01)
significant_features = feature_importances_df[feature_importances_df['Importance'] > 0.01]
print("Significant features:")
print(significant_features)

stop !

## Compare distributions

In [ ]:
TAFT = cgt.TAFT_x_df.copy()
TAFT['whether_TAFT'] = 1
GUAT = cgt.GUAT_x_df.copy()
GUAT['whether_TAFT'] = 0
both_df = pd.concat([TAFT, GUAT], axis=0)

In [ ]:

# for each feature in significant_features, plot the histogram of the feature for each class
for feature in ['cur_ff_distance_at_ref', 'cur_ff_angle_at_ref']:
    sns.histplot(x=feature, data=both_df, stat='probability', kde=False, hue='whether_TAFT', common_norm=False)
    plt.title(f'{feature} histogram')
    plt.show()

In [ ]:
# for each feature in significant_features, plot the histogram of the feature for each class
max_features_to_plot = 3
count = 0
for feature in significant_features['Feature']:
    sns.histplot(x=feature, data=both_df, stat='probability', kde=False, hue='whether_TAFT', common_norm=False)
    plt.title(f'{feature} histogram')
    plt.show()
    count += 1
    if count >= max_features_to_plot:
        break

### exp

In [ ]:
cgt.TAFT_trials_df

In [ ]:
cgt.GUAT_w_ff_df

### plot trials

In [ ]:
GUAT_or_TAFT = 'GUAT'
list_of_stop_point_index = cgt.GUAT_x_df[cgt.GUAT_x_df['cur_ff_distance_at_ref'] > 150].stop_point_index.values

In [ ]:
GUAT_or_TAFT = 'TAFT'
list_of_stop_point_index = cgt.TAFT_x_df[cgt.TAFT_x_df['cur_ff_distance_at_ref'] > 150].stop_point_index.values

In [ ]:
sub = cgt.take_out_subsets_to_plot(list_of_stop_point_index, GUAT_or_TAFT)

In [ ]:
counter = 0
for i, row in sub.iterrows():
    if GUAT_or_TAFT == 'GUAT':
        ff_column = 'latest_visible_ff'
    else:
        ff_column = 'ff_index'

    print('cur_ff_distance_at_ref', row['cur_ff_distance_at_ref'])
    additional_kwargs = {'point_indices_to_be_marked': row['stop_point_index'],
                         'point_indices_to_be_marked_3rd_kind': row['ref_point_index']
                         }
    
    plotting_kwargs_temp = copy.deepcopy(cgt.plotting_kwargs)
    plotting_kwargs_temp['indices_of_ff_to_mark'] = [row[ff_column]]
    trial = row['trial']
    duration = [cgt.ff_caught_T_new[trial-2], cgt.ff_caught_T_new[trial]+1]
    returned_info = plot_trials.PlotTrials(duration, 
                *cgt.PlotTrials_args,
                **plotting_kwargs_temp,   
                **additional_kwargs          
                )
    plt.show()
    # process_GUAT_trials_class.ProcessGUATtrials.make_GUAT_plot(cgt, trial, [row[ff_column]], np.arange(row['stop_point_index']-200, row['stop_point_index']+300),
    #                                                            additional_kwargs=additional_kwargs)
    if counter == 3:
        break
    counter += 1